In [42]:
import sys
sys.path.append("../")
import os
import pandas as pd
from tqdm import tqdm
import webrtcvad
from utils.basic_utils import make_valid_path
from utils.audio_utils import remove_silence, byte_to_float, get_wav_len

In [43]:
esc_raw_path_in = '/home/alhasan/workdir/SED/data/ESC-50/raw/audio_16k'

In [44]:
# read csv
esc_50 = pd.read_csv('/home/alhasan/workdir/SED/data/ESC-50/esc50.csv')
# extract esc10 files
esc_10 = esc_50.loc[esc_50['esc10'] == True].reset_index()[['filename', 'fold', 'category']].rename({'category': 'event'}, axis=1)
events = esc_10['event'].unique().tolist()
esc_10['event_id'] = esc_10['event'].apply(events.index)
esc_10['esc10'] = True
# save csv
esc_10.to_csv('/home/alhasan/workdir/SED/data/ESC-50/esc10.csv')

In [45]:
esc_10.head(5)

,filename,fold,event,event_id,esc10
0,1-100032-A-0.wav,1,dog,0,True
1,1-110389-A-0.wav,1,dog,0,True
2,1-116765-A-41.wav,1,chainsaw,1,True
3,1-17150-A-12.wav,1,crackling_fire,2,True
4,1-172649-A-40.wav,1,helicopter,3,True


In [46]:
esc_raw_path_out = '/home/alhasan/workdir/SED/data/ESC-50/raw/no_silence/audio_16k'
make_valid_path(esc_raw_path_out, is_dir=True, exist_ok=True)
short_files = []
for i in tqdm(range(len(esc_10))):
    audio_name = esc_10.iloc[i]['filename']
    audio_path_in = os.path.join(esc_raw_path_in, audio_name)
    audio_path_out = os.path.join(esc_raw_path_out, audio_name)
    audio_collected, sample_rate = remove_silence(audio_path_in, audio_path_out, frame_duration_ms = 20,padding= 100, vad_level=0)
    float_audio = byte_to_float(audio_collected)
    wave_len =  get_wav_len(float_audio, sample_rate)
    if wave_len < 2500:
        short_files.append(audio_name)

print("short files count ", len(short_files))
print("short files: ", short_files)

 

100%|██████████| 400/400 [00:00<00:00, 480.65it/s]

short files count  66
short files:  ['1-100032-A-0.wav', '1-110389-A-0.wav', '1-26143-A-21.wav', '1-27724-A-1.wav', '1-29680-A-21.wav', '1-31748-A-21.wav', '1-34119-A-1.wav', '1-34119-B-1.wav', '1-44831-A-1.wav', '1-47273-A-21.wav', '1-47274-A-21.wav', '1-54505-A-21.wav', '1-59324-A-21.wav', '1-81883-A-21.wav', '2-109505-A-21.wav', '2-118072-A-0.wav', '2-118964-A-0.wav', '2-119102-A-21.wav', '2-128631-A-21.wav', '2-130978-A-21.wav', '2-130979-A-21.wav', '2-135728-A-38.wav', '2-141584-A-38.wav', '2-28314-A-12.wav', '2-28314-B-12.wav', '2-65750-A-1.wav', '2-82538-A-21.wav', '2-95258-A-1.wav', '2-96460-A-1.wav', '3-107219-A-1.wav', '3-137152-A-1.wav', '3-142601-A-21.wav', '3-142605-A-21.wav', '3-143119-A-21.wav', '3-148330-A-21.wav', '3-149189-A-1.wav', '3-150231-A-21.wav', '3-154926-A-40.wav', '3-156558-A-21.wav', '3-157187-A-12.wav', '3-163459-A-0.wav', '3-170015-A-0.wav', '3-170377-A-38.wav', '3-171012-A-38.wav', '3-187710-A-11.wav', '4-156843-A-21.wav', '4-161579-A-40.wav', '4-164064-

### Split esc-10 to two classes

In [47]:
esc10 = pd.read_csv('/home/alhasan/workdir/SED/data/ESC-50/esc10.csv')
grp_h = ['helicopter', 'rain', 'sea_waves', 'crackling_fire', 'chainsaw']

In [48]:
esc10_H = esc_10.loc[esc10['event'].isin(grp_h)]
esc_h_labels = list(esc10_H['event_id'].unique())
esc10_H['event_id_5'] = esc10_H['event_id'].apply(esc_h_labels.index)

/tmp/ipykernel_1201104/1391282168.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  esc10_H['event_id_5'] = esc10_H['event_id'].apply(esc_h_labels.index)


In [49]:
esc10_H.head(3)

,filename,fold,event,event_id,esc10,event_id_5
2,1-116765-A-41.wav,1,chainsaw,1,True,0
3,1-17150-A-12.wav,1,crackling_fire,2,True,1
4,1-172649-A-40.wav,1,helicopter,3,True,2


In [50]:
esc10_P = esc_10.loc[~esc10['event'].isin(grp_h)]
esc_p_labels = list(esc10_P['event_id'].unique())
esc10_P['event_id_5'] = esc10_P['event_id'].apply(esc_p_labels.index)

/tmp/ipykernel_1201104/70753375.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  esc10_P['event_id_5'] = esc10_P['event_id'].apply(esc_p_labels.index)


In [51]:
esc10_P.head(3)

,filename,fold,event,event_id,esc10,event_id_5
0,1-100032-A-0.wav,1,dog,0,True,0
1,1-110389-A-0.wav,1,dog,0,True,0
17,1-187207-A-20.wav,1,crying_baby,5,True,1


In [52]:
esc10_H.to_csv('/home/alhasan/workdir/SED/data/ESC-50/esc10_H.csv')
esc10_P.to_csv('/home/alhasan/workdir/SED/data/ESC-50/esc10_P.csv')
